### Q1 Running Elastic

I am running a different version of elastic search (9.0.2)

In [1]:
from elasticsearch import Elasticsearch


es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'bc38554e653a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tchAMYDXSSK_5tfaVFHnBw', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

### Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2 Indexing the Data

In [5]:
from tqdm.auto import tqdm

es_client = Elasticsearch('http://localhost:9200')
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


index_name = 'assignment-questions'
es_client.indices.create(index=index_name, body=index_settings)

# Add data using the index method
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:03<00:00, 289.35it/s]


### Q3 Searching

In [6]:
query = 'How do execute a command on a Kubernetes pod?'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
print(response)

{'took': 17, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'assignment-questions', '_id': 'RQyiWpcBBSEADN1cSEqP', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'assignment-questions', '_id': '1AyiWpcBBSEADN1cSko7', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashb

### Q4 Filtering

In [7]:
query = 'How do copy a file to a Docker container?'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    
    
print(result_docs[2]['question'])

    

How do I copy files from a different folder into docker container’s working directory?


### Q5 Building a prompt

In [22]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
for doc in result_docs[:3]:
    context += context_template.format(question=doc['question'], text=doc['text'])
    context += "\n\n"    
context = context.rstrip('\n')

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context)
# print(prompt)
len(prompt)

1446

### Q6 Tokens

In [33]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
encoded = encoding.encode(prompt)
len(encoded)


320

### Ungraded

In [34]:
from openai import OpenAI
from dotenv import load_dotenv
import os


load_dotenv()


llm_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
llm_response = llm_client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': prompt}]
)

In [35]:
llm_response.choices[0].message.content

'You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is as follows:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```\n\nThis command copies a file or directory from your local machine into a running Docker container at the specified path.'